In [1]:
!pip3 install -q langchain 
!pip3 install -q arize 
!pip3 install -q 'arize[AutoEmbeddings]'
!pip3 install -q "openai<1.0.0"


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks.arize_callback import ArizeCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.llms import OpenAI

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()

API_KEY = os.environ["API_KEY"]
SPACE_KEY = os.environ["SPACE_KEY"]

if SPACE_KEY == "YOUR_SPACE_KEY" or API_KEY == "YOUR_API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")

# Define callback handler for Arize
arize_callback = ArizeCallbackHandler(
    model_id="llm-langchain-demo",
    model_version="1.0",
    SPACE_KEY=SPACE_KEY,
    API_KEY=API_KEY,
)

  arize.utils.logging | WARNING | No available GPU has been detected. The use of GPU acceleration is strongly recommended. You can check for GPU availability by running `torch.cuda.is_available()`
  arize.utils.logging | INFO | Downloading pre-trained model 'distilbert-base-uncased'
  arize.utils.logging | INFO | Downloading tokenizer for 'distilbert-base-uncased'
✅ Arize client setup done! Now you can start using Arize!


In [5]:
manager = CallbackManager([StdOutCallbackHandler(), arize_callback])
llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)

In [6]:
llm_result = llm.generate(
    [
        "Tell me an interesting fact about pandas.",
        "Explain the concept of overfitting in 2 sentences.",
    ]
)

  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00,  5.27 examples/s]


  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00, 19.76 examples/s]


  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo1MTU0/spaces/U3BhY2U6NTQ0Nw==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion
✅ Successfully logged data to Arize!
  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00, 19.47 examples/s]


  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00, 17.91 examples/s]


  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo1MTU0/spaces/U3BhY2U6NTQ0Nw==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion
✅ Successfully logged data to Arize!


In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, callback_manager=manager)

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True, callback_manager=manager
)

test_prompts = [
    {
        "input": "documentary about pandas who are about be extinct because of global warming"
    },
    {"input": "once upon a time in hollywood"},
    {"input": "the best mo observability tooling"},
]
overall_chain.apply(test_prompts)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: documentary about pandas who are about be extinct because of global warming
Playwright: This is a synopsis for the above play:
  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00, 18.47 examples/s]


  arize.utils.logging | INFO | Generating embedding vectors


Map: 100%|██████████| 1/1 [00:00<00:00, 14.05 examples/s]


  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo1MTU0/spaces/U3BhY2U6NTQ0Nw==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion
✅ Successfully logged data to Arize!

> Finished chain.


Documentary about Pandas Who Are About to Be Extinct Because of Global Warming is a play that follows the story of a group of pandas living in a remote mountain region of China. As the climate changes, their habitat is slowly being destroyed, and they are facing extinction. The play follows the pandas as they struggle to survive in their rapidly changing environment, and the people who are tryi

...

Map: 100%|██████████| 1/1 [00:00<00:00, 13.84 examples/s]


  arize.utils.logging | WARNING | prediction_label_column_name was not provided, a default prediction label equal to 1 will be set for GENERATIVE_LLM models.
  arize.utils.logging | WARNING | actual_label_column_name was not provided. Some metrics that require actual labels, e.g. correctness or accuracy, may not be computed.
  arize.utils.logging | INFO | Success! Check out your data at https://app.arize.com/organizations/QWNjb3VudE9yZ2FuaXphdGlvbjo1MTU0/spaces/U3BhY2U6NTQ0Nw==/models/modelName/llm-langchain-demo?selectedTab=dataIngestion


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for text-davinci-003 in organization org-hlc7lWO9D9K29sQNyGlPfR55 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


✅ Successfully logged data to Arize!

> Finished chain.


Once Upon a Time in Hollywood is a story about a group of struggling actors and filmmakers in the 1960s who are trying to make it big in the entertainment industry. The play follows the lives of these characters as they navigate the highs and lows of the industry, from the glamour of the red carpet to the harsh realities of rejection and failure. Along the way, they must confront their own insecurities and doubts, as well as the challenges of a rapidly changing world. As they strive to make their dreams come true, they must also grapple with the consequences of their choices and the power of the Hollywood machine. Ultimately, the play is a story of hope and resilience, as the characters learn to embrace their own unique paths and find success in their own way.


> Entering new LLMChain chain...
Prompt after formatting:
You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for

RateLimitError: Rate limit reached for text-davinci-003 in organization org-hlc7lWO9D9K29sQNyGlPfR55 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.